In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse

In [2]:
# 期間の算術について調べる
# Periodオブジェクト
# 下の例は2007/1/1から2007/12/31までを含んでいる
# A: year end frequency
p = pd.Period(2007, freq='A-DEC')
p

Period('2007', 'A-DEC')

In [3]:
p + 5

Period('2012', 'A-DEC')

In [4]:
p - 2

Period('2005', 'A-DEC')

In [5]:
# 7年分のずれを算出
pd.Period('2014', freq='A-DEC') - p

7

In [6]:
# 定期的な範囲の期間はperiod_rangeを使う
# PeriodIndexクラスはPeriodのシーケンス
rng = pd.period_range('1/1/2000', '6/30/2000', freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

In [7]:
# PeriodIndexクラスはpandasのデータ構造の中でindexとして使える
pd.Series(np.random.randn(6), index=rng)

2000-01   -0.367617
2000-02    0.223291
2000-03   -0.535391
2000-04   -1.071273
2000-05   -0.784209
2000-06   -0.016882
Freq: M, dtype: float64

In [8]:
# 文字列の配列を時系列にしたいときにもPeriodIndexオブジェクトが使える
values = ['2001Q3', '2002Q2', '2003Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

In [9]:
# PeriodやPeriodIndexオブジェクトはasfreqメソッドを使って別の頻度に変換できる
# how : {‘start’, ‘end’}, default end
# 下の例は2007年の12月が終点の年頻度を月頻度に変換している
p = pd.Period('2007', freq='A-DEC')
p.asfreq('M', how='start')

Period('2007-01', 'M')

In [10]:
# 年期間を月期間に変換するときは、始点か終点を指定しなければいけない
p.asfreq('M', how='end')

Period('2007-12', 'M')

In [11]:
# 2007年の6月が終点の年単位期間を、月単位期間に
p = pd.Period('2007', freq='A-JUN')
p.asfreq('M', 'start')

Period('2006-07', 'M')

In [12]:
p.asfreq('M', 'end')

Period('2007-06', 'M')

In [13]:
# 高い頻度(月単位の頻度)から低い頻度(年単位の頻度)に変換するときは、短い方の期間がどこに含まれてるかで変換後の期間が決まる
p = pd.Period('2007-08', 'M')
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

In [14]:
# Periodの頻度変換の考え方はPeriodIndexの頻度変換でも同じ考え方で操作できる
rng = pd.period_range('2006', '2009', freq='A-DEC')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2006    1.209583
2007    1.022539
2008    0.813612
2009    0.148853
Freq: A-DEC, dtype: float64

In [15]:
ts.asfreq('M', how='start')

2006-01    1.209583
2007-01    1.022539
2008-01    0.813612
2009-01    0.148853
Freq: M, dtype: float64

In [16]:
# B: business day frequency
ts.asfreq('B', how='end')

2006-12-29    1.209583
2007-12-31    1.022539
2008-12-31    0.813612
2009-12-31    0.148853
Freq: B, dtype: float64

In [17]:
# Q: quarter end frequency
p = pd.Period('2012Q4', freq='Q-JAN')
p

Period('2012Q4', 'Q-JAN')

In [18]:
# D: day
# 頻度を日にちに変換
# startで四半期の始点を指定
p.asfreq('D', 'start')

Period('2011-11-01', 'D')

In [19]:
# endを指定して四半期の最終日を指定
p.asfreq('D', 'end')

Period('2012-01-31', 'D')

In [20]:
# 四半期の最後の営業日('B', 'e')の一日前の16時のタイムスタンプを取得
p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
p4pm

Period('2012-01-30 16:00', 'T')

In [21]:
# タイムスタンプに変換
p4pm.to_timestamp()

Timestamp('2012-01-30 16:00:00')

In [22]:
# 四半期を使った一定の範囲を生成するときはこう
rng = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')
ts = pd.Series(np.arange(len(rng)), index=rng)
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int64

In [23]:
# 四半期の最後の営業日('B', 'e')の一日前の16時のタイムスタンプをインデックスにする
new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
ts.index = new_rng.to_timestamp()
ts

2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int64

In [24]:
rng = pd.date_range('1/1/2000', periods=3, freq='M')
ts = pd.Series(np.random.randn(3), index=rng)
ts

2000-01-31   -1.527158
2000-02-29   -0.327313
2000-03-31   -0.223335
Freq: M, dtype: float64

In [25]:
# タイムスタンプのインデックスを期間(period)に変換する
pts = ts.to_period()
pts

2000-01   -1.527158
2000-02   -0.327313
2000-03   -0.223335
Freq: M, dtype: float64

In [26]:
# 新しいPeriodIndexはタイムスタンプから推論されるが、任意の頻度を指定することも出来る
rng = pd.date_range('1/29/2000', periods=6, freq='D')
ts2 = pd.Series(np.random.randn(6), index=rng)
ts2.to_period('M')

2000-01    1.917397
2000-01    0.841772
2000-01   -0.635472
2000-02   -0.337218
2000-02   -1.022222
2000-02   -0.065696
Freq: M, dtype: float64

In [27]:
pts = ts.to_period()
pts

2000-01   -1.527158
2000-02   -0.327313
2000-03   -0.223335
Freq: M, dtype: float64

In [28]:
# Periodをタイムスタンプに戻したいときはto_timestamp
pts.to_timestamp(how='end')

2000-01-31   -1.527158
2000-02-29   -0.327313
2000-03-31   -0.223335
Freq: M, dtype: float64

In [29]:
# 下のデータセットについて考える
data = pd.read_csv('../data_wrangling/macrodata.csv')
data.year[:10]

0    1959.0
1    1959.0
2    1959.0
3    1959.0
4    1960.0
5    1960.0
6    1960.0
7    1960.0
8    1961.0
9    1961.0
Name: year, dtype: float64

In [30]:
data.quarter[:10]

0    1.0
1    2.0
2    3.0
3    4.0
4    1.0
5    2.0
6    3.0
7    4.0
8    1.0
9    2.0
Name: quarter, dtype: float64

In [31]:
# 上の年度と四半期のインデックスを統合したい
# PeriodIndexクラスに頻度を指定することで、データフレーム用のインデックスとして統合することが出来る
index = pd.PeriodIndex(year=data.year, quarter=data.quarter, freq='Q-DEC')
index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')

In [32]:
data.index = index
data.infl[:10]

1959Q1    0.00
1959Q2    2.34
1959Q3    2.74
1959Q4    0.27
1960Q1    2.31
1960Q2    0.14
1960Q3    2.70
1960Q4    1.21
1961Q1   -0.40
1961Q2    1.47
Freq: Q-DEC, Name: infl, dtype: float64